In [1]:
ct_model='/home/nada.saadi/CTPET/hecktor2022_cropped/mda-ctonly-PT/ctonly-PT-with4.pth'
pet_model='/home/nada.saadi/CTPET/hecktor2022_cropped/mda-ptonly-PT/ptonly-PT-with4.pth'

# ct_model_6='/home/nada.saadi/CTPET/hecktor2022_cropped/6thctonlychannel_PRET/6thCTonly_channel_pret.pth'
# pet_model_6='/home/nada.saadi/CTPET/hecktor2022_cropped/6th_PET_channel_PRET/6thPETonly_channel_pret.pth'

In [2]:
import os
import shutil
import tempfile

import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from glob import glob
# import wandb

import monai
from monai.losses import DiceCELoss, DiceFocalLoss, FocalLoss
from monai.inferers import sliding_window_inference
from monai import transforms

from monai.transforms import (
    AsDiscrete,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandFlipd,
    RandCropByPosNegLabeld,
    RandShiftIntensityd,
    ScaleIntensityRanged,
    Spacingd,
    RandRotate90d,
    MapTransform,
    ScaleIntensityd,
    #AddChanneld,
    SpatialPadd,
    CenterSpatialCropd,
    EnsureChannelFirstd,
    ConcatItemsd,
    AdjustContrastd, 
    Rand3DElasticd,
    HistogramNormalized,
    NormalizeIntensityd,
    Invertd,
    SaveImage,

)

from monai.config import print_config
from monai.metrics import DiceMetric
from monai.networks.nets import SwinUNETR, UNETR, SegResNet

from monai.data import (
    DataLoader,
    CacheDataset,
    load_decathlon_datalist,
    decollate_batch,
)
from monai import data


from monai.utils import first, set_determinism
from sklearn.model_selection import train_test_split
import json


from monai.transforms import apply_transform
from monai.inferers import sliding_window_inference
from monai.transforms import AsDiscrete
from monai.data import DataLoader, Dataset, decollate_batch
from tqdm import tqdm
import json
import torch

In [3]:


os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load the pretrained weights
#pretrained_weights = torch.load('/home/nada.saadi/CTPET/hecktor2022_cropped/4centers-ctonly/4centers-ctonly.pth')


model_ct = UNETR(
    in_channels=1,
    out_channels=3,
    img_size=(96, 96, 96),
    feature_size=16,
    hidden_size=768,
    mlp_dim=3072, 
    num_heads=12,
    pos_embed="perceptron",
    norm_name="instance",
    res_block=True,
    dropout_rate=0.0,
).to(device)

model_pet = UNETR(
    in_channels=1,
    out_channels=3,
    img_size=(96, 96, 96),
    feature_size=16,
    hidden_size=768,
    mlp_dim=3072, 
    num_heads=12,
    pos_embed="perceptron",
    norm_name="instance",
    res_block=True,
    dropout_rate=0.0,
).to(device)

/home/nada.saadi/miniconda3/envs/clam/lib/python3.8/site-packages/monai/utils/deprecate_utils.py:221: FutureWarning: monai.networks.nets.unetr UNETR.__init__:pos_embed: Argument `pos_embed` has been deprecated since version 1.2. It will be removed in version 1.4. please use `proj_type` instead.
  warn_deprecated(argname, msg, warning_category)


In [4]:
model_ct.load_state_dict(torch.load(ct_model))
model_pet.load_state_dict(torch.load(pet_model))

model_ct.eval() 
model_pet.eval()


UNETR(
  (vit): ViT(
    (patch_embedding): PatchEmbeddingBlock(
      (patch_embeddings): Sequential(
        (0): Rearrange('b c (h p1) (w p2) (d p3) -> b (h w d) (p1 p2 p3 c)', p1=16, p2=16, p3=16)
        (1): Linear(in_features=4096, out_features=768, bias=True)
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (blocks): ModuleList(
      (0-11): 12 x TransformerBlock(
        (mlp): MLPBlock(
          (linear1): Linear(in_features=768, out_features=3072, bias=True)
          (linear2): Linear(in_features=3072, out_features=768, bias=True)
          (fn): GELU(approximate='none')
          (drop1): Dropout(p=0.0, inplace=False)
          (drop2): Dropout(p=0.0, inplace=False)
        )
        (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): SABlock(
          (out_proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_q): Linear(in_features=768, out_features=768, bias=False)
          (proj_k): Linear(in_features=

In [5]:
ct_data='/home/nada.saadi/CTPET/hecktor2022_cropped/test_ct_only_mda.json'
pet_data='/home/nada.saadi/CTPET/hecktor2022_cropped/test_pet_only_mda.json'

# ct_data_6='/home/nada.saadi/CTPET/hecktor2022_cropped/HGJ_CT_train_new.json'
# pet_data_6='/home/nada.saadi/CTPET/hecktor2022_cropped/HGJ_PET_train_new.json'

In [6]:
import json

# Path to the existing JSON file
original_json_path = ct_data
# Path for the new simplified JSON file
new_json_path = '/home/nada.saadi/CTPET/hecktor2022_cropped/simplifiedct_json_file.json'

# Read the original JSON file
with open(original_json_path, 'r') as file:
    data = json.load(file)

# Modify each dictionary in the list to retain only 'ct' and 'seg' keys
simplified_data = [{'ct': item['ct'], 'seg': item['seg']} for item in data if 'ct' in item and 'seg' in item]

# Save the modified list of dictionaries to a new JSON file
with open(new_json_path, 'w') as file:
    json.dump(simplified_data, file, indent=4)

print(f"Simplified JSON file has been saved to {new_json_path}")


Simplified JSON file has been saved to /home/nada.saadi/CTPET/hecktor2022_cropped/simplifiedct_json_file.json


In [14]:
# import json

# # Path to the existing JSON file
# original_json_path = '/home/nada.saadi/CTPET/hecktor2022_cropped/HGJ_CT_test_new.json'
# # Path for the new simplified JSON file
# new_json_path = '/home/nada.saadi/CTPET/hecktor2022_cropped/HGJ_ct_test_data_simplified_json_file.json'

# # Read the original JSON file
# with open(original_json_path, 'r') as file:
#     data = json.load(file)

# # Modify each dictionary in the list to retain only 'ct' and 'seg' keys
# simplified_data = [{'pt': item['pt'], 'seg': item['seg']} for item in data if 'pt' in item and 'seg' in item]

# # Save the modified list of dictionaries to a new JSON file
# with open(new_json_path, 'w') as file:
#     json.dump(simplified_data, file, indent=4)

# print(f"Simplified JSON file has been saved to {new_json_path}")

FileNotFoundError: [Errno 2] No such file or directory: '/home/nada.saadi/CTPET/hecktor2022_cropped/HGJ_CT_test_new.json'

In [7]:
# import json

# # Path to the train JSON file
# #train_json_path = '/home/nada.saadi/CTPET/hecktor2022_cropped/HGJ_CT_train_new.json'
# train_json_path = '/home/nada.saadi/CTPET/hecktor2022_cropped/HGJ_PET_train_new.json'
# # Path for the new test JSON file
# test_json_path = '/home/nada.saadi/CTPET/hecktor2022_cropped/HGJ_pet_test.json'

# # Read the train JSON file
# with open(train_json_path, 'r') as file:
#     train_data = json.load(file)

# # Filter the train data to get only the test files
# test_data = [item for item in train_data if 'validation' in item and item['validation']]

# # Save the test data to a new JSON file
# with open(test_json_path, 'w') as file:
#     json.dump(test_data, file, indent=4)

# print(f"Test JSON file has been saved to {test_json_path}")


Test JSON file has been saved to /home/nada.saadi/CTPET/hecktor2022_cropped/HGJ_pet_test.json


In [7]:
# ct_test_data_6='/home/nada.saadi/CTPET/hecktor2022_cropped/HGJ_CT_test_new.json'
# pet_test_data_6='/home/nada.saadi/CTPET/hecktor2022_cropped/HGJ_pet_test_new.json'

In [8]:
ct_data_used_simplified='/home/nada.saadi/CTPET/hecktor2022_cropped/simplifiedct_json_file.json'
pet_data_used_simplified='/home/nada.saadi/CTPET/hecktor2022_cropped/simplifiedpt_json_file.json'

In [9]:
# import json

# # Open the JSON file
# with open('/home/nada.saadi/CTPET/hecktor2022_cropped/simplifiedpt_json_file.json') as f:
#     data = json.load(f)

# # Print the first element
# print(data[0])

In [10]:
# def check_data_format(data):
#     if not isinstance(data, list):
#         print("Data is not a list.")
#         return False

#     for i, item in enumerate(data):
#         if not isinstance(item, dict):
#             print(f"Item at index {i} is not a dictionary.")
#             return False
#         if len(item) != 2:
#             print(f"Dictionary at index {i} does not have exactly two key-value pairs.")
#             return False

#     print("Data format is correct.")
#     return True

# # Check the format of valid_data
# check_data_format(ct_data)

In [11]:
# def create_modified_data(ct_data):
#     modified_data = []
#     for item in ct_data:
#         modified_item = {key: item[key] for key in item.keys()}
#         modified_data.append(modified_item)
#     return modified_data

# # Create a modified version of valid_data
# modified_valid_data = create_modified_data(new_json_path)

# # Check the format of modified_valid_data
# check_data_format(modified_valid_data)

In [9]:
class ClipCT(MapTransform):
    """
    Convert labels to multi channels based on hecktor classes:
    label 1 is the tumor
    label 2 is the lymph node

    """

    def __call__(self, data):
        d = dict(data)
        for key in self.keys:
            if key == "ct":
                d[key] = torch.clip(d[key], min=-200, max=200)
            # elif key == "pt":
            #     d[key] = torch.clip(d[key], d[key].min(), 5)
        return d

class MulPTFM(MapTransform):
    """
    Mult PT and FM 

    """

    def __call__(self, data):
        d = dict(data)

        fm = d["ct"] > 0
        d["pt"] = d["pt"] * fm
        return d

class SelectClass(MapTransform):
    """
    Select the class for which you want to fine tune the model 

    """
    # def __init__(self, keys, cls=1):
    #     super(self).__init__(keys)
    #     self.cls = cls

    def __call__(self, data):
        d = dict(data)
        d["seg"][d["seg"] == 1] = 0
        # d["seg"][d["seg"] == 2] = 1
        
        return d

In [10]:
val_transforms_ct = Compose(
    [
        LoadImaged(keys=["ct",  "seg"], ensure_channel_first = True),
        SpatialPadd(keys=["ct",  "seg"], spatial_size=(200, 200, 310), method='end'),
        Orientationd(keys=["ct",  "seg"], axcodes="PLS"),
        #NormalizeIntensityd(keys=["pt"]),
        ClipCT(keys=["ct"]),
        ScaleIntensityd(keys=["ct",], minv=0, maxv=1),
        #MulPTFM(keys=["ct","pt"]),
        ConcatItemsd(keys=[ "ct"], name="ct"),
    ]
)
def create_dataloader(data, transforms, batch_size=2, shuffle=True):
    # Create CacheDataset with the reformatted data
    dataset = Dataset(data=data, transform=transforms)

    # Create DataLoader
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=8)
val_loader = create_dataloader(ct_data, val_transforms_ct, shuffle=False)

In [14]:
import json

# Path to the JSON file
json_file_path = '/home/nada.saadi/CTPET/hecktor2022_cropped/simplifiedct_json_file.json'

# Read the JSON file
with open(json_file_path, "r") as file:
    ct_test_data = json.load(file)

# Access the data from the JSON file
# Example: Print the first element
print(ct_test_data[0])


{'ct': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-001/MDA-001_ct.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-001/MDA-001_gt.nii.gz'}


In [15]:
import io

with open(json_dir, 'r') as f:
    test_data = json.load(f)

test_loader =DataLoader(
    dataset=Dataset(data=test_data_ct, transform=val_transforms_ctpt),
    batch_size=1,  # Batch size for testing can be 1 since no backpropagation is required
    shuffle=False,
    num_workers=4
)

NameError: name 'json_dir' is not defined

In [16]:
# import io

# with open(json_dir, 'r') as f:
#     test_data = json.load(f)

test_loader = DataLoader(
    dataset=Dataset(data=ct_test_data, transform=val_transforms_ct),
    batch_size=1,  # Batch size for testing can be 1 since no backpropagation is required
    shuffle=False,
    num_workers=4
)

loss_function = DiceCELoss(to_onehot_y=True, softmax=True)
optimizer = torch.optim.AdamW(model_ct.parameters(), lr=1e-4, weight_decay=1e-5)

post_label = AsDiscrete(to_onehot=3)
post_pred = AsDiscrete(argmax=True, to_onehot=3)

# Dice metric for evaluation
dice_metric_fn = DiceMetric(include_background=False, reduction="mean", get_not_nans=False)
dice_metric_batch_fn = DiceMetric(include_background=False, reduction="mean_batch")

pt_weights='/home/nada.saadi/CTPET/hecktor2022_cropped/mda-ctonly-PT/ctonly-PT-with4.pth'
model_ct.load_state_dict(torch.load(pt_weights))
def testing():
    model_ct.eval()
    dice_metric_fn.reset()
    dice_metric_batch_fn.reset()
    with torch.no_grad():
        for batch_data in tqdm(test_loader):
            test_inputs, test_labels = batch_data["ct"].cuda(), batch_data["seg"].cuda()
            
            test_outputs_ct = sliding_window_inference(test_inputs, (96, 96, 96), 4, model_ct)
            
            # Convert outputs and labels to one-hot format for DiceMetric calculation
            test_outputs_ct = [AsDiscrete(argmax=True, to_onehot=3)(i) for i in decollate_batch(test_outputs_ct)]
            test_labels = [AsDiscrete(to_onehot=3)(i) for i in decollate_batch(test_labels)]

            dice_metric_fn(y_pred=test_outputs_ct, y=test_labels)
            dice_metric_batch_fn(y_pred=test_outputs_ct, y=test_labels)
            
        # After processing all batches, save the outputs
    #ct_save_path = '/home/nada.saadi/CTPET/hecktor2022_cropped/test_output_ct_saved.pt'
    ct_save_path_6thcenter = '/home/nada.saadi/CTPET/hecktor2022_cropped/HGJ_test_output_ct_saved.pt'
    torch.save(test_outputs_ct, ct_save_path_6thcenter)
    print(f"Outputs saved at: {ct_save_path_6thcenter}")
    

    mean_dice_test = dice_metric_fn.aggregate().item()
    metric_batch_test = dice_metric_batch_fn.aggregate()
    metric_tumor = metric_batch_test[0].item()
    metric_lymph = metric_batch_test[1].item()

    print(f"Testing - Avg Dice: {mean_dice_test:.4f}, Tumor Dice: {metric_tumor:.4f}, Lymph Dice: {metric_lymph:.4f}")
    return mean_dice_test, metric_tumor, metric_lymph
    # Save the weights
testing()

100%|██████████| 44/44 [00:27<00:00,  1.58it/s]


Outputs saved at: /home/nada.saadi/CTPET/hecktor2022_cropped/HGJ_test_output_ct_saved.pt
Testing - Avg Dice: 0.4338, Tumor Dice: 0.3914, Lymph Dice: 0.4983


(0.4337661862373352, 0.3913649022579193, 0.49833863973617554)

In [18]:
import json

# Path to the JSON file
json_file_path = '/home/nada.saadi/CTPET/hecktor2022_cropped/simplifiedpt_json_file.json'

# Read the JSON file
with open(json_file_path, "r") as file:
    pt_test_data = json.load(file)

# Access the data from the JSON file
# Example: Print the first element
print(pt_test_data[0])

{'pt': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-005/MDA-005_pt.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-005/MDA-005_gt.nii.gz'}


In [19]:
val_transforms_pt = Compose(
    [
        LoadImaged(keys=[ "pt", "seg"], ensure_channel_first = True),
        SpatialPadd(keys=[ "pt", "seg"], spatial_size=(200, 200, 310), method='end'),
        Orientationd(keys=[ "pt", "seg"], axcodes="PLS"),
        NormalizeIntensityd(keys=["pt"]),
        ClipCT(keys=["pt"]),
        ScaleIntensityd(keys=["pt"], minv=0, maxv=1),
        #MulPTFM(keys=["ct","pt"]),
        ConcatItemsd(keys=["pt"], name="pt"),
    ]
)
def create_dataloader(data, transforms, batch_size=2, shuffle=True):
    # Create CacheDataset with the reformatted data
    dataset = Dataset(data=data, transform=transforms)

    # Create DataLoader
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=8)
val_loader = create_dataloader(pt_test_data, val_transforms_pt, shuffle=False)

In [20]:
test_loader = DataLoader(
    dataset=Dataset(data=pt_test_data, transform=val_transforms_pt),
    batch_size=1,  # Batch size for testing can be 1 since no backpropagation is required
    shuffle=False,
    num_workers=4
)

loss_function = DiceCELoss(to_onehot_y=True, softmax=True)
optimizer = torch.optim.AdamW(model_pet.parameters(), lr=1e-4, weight_decay=1e-5)

post_label = AsDiscrete(to_onehot=3)
post_pred = AsDiscrete(argmax=True, to_onehot=3)

# Dice metric for evaluation
dice_metric_fn = DiceMetric(include_background=False, reduction="mean", get_not_nans=False)
dice_metric_batch_fn = DiceMetric(include_background=False, reduction="mean_batch")

# pt_weights='/home/nada.saadi/CTPET/hecktor2022_cropped/4centers-ctonly/4centers-ctonly.pth'
# model_ct.load_state_dict(torch.load(pt_weights))
def testing():
    model_ct.eval()
    dice_metric_fn.reset()
    dice_metric_batch_fn.reset()
    with torch.no_grad():
        for batch_data in tqdm(test_loader):
            test_inputs, test_labels = batch_data["pt"].cuda(), batch_data["seg"].cuda()
            
            test_outputs_pt = sliding_window_inference(test_inputs, (96, 96, 96), 4, model_pet)
            
            # Convert outputs and labels to one-hot format for DiceMetric calculation
            test_outputs_pt = [AsDiscrete(argmax=True, to_onehot=3)(i) for i in decollate_batch(test_outputs_pt)]
            test_labels = [AsDiscrete(to_onehot=3)(i) for i in decollate_batch(test_labels)]

            dice_metric_fn(y_pred=test_outputs_pt, y=test_labels)
            dice_metric_batch_fn(y_pred=test_outputs_pt, y=test_labels)



    # After processing all batches, save the outputs
    pt_save_path = '/home/nada.saadi/CTPET/hecktor2022_cropped/test_outputs_pt_saved.pt'
    torch.save(test_outputs_pt, pt_save_path)
    print(f"Outputs saved at: {pt_save_path}")
    
    
    mean_dice_test = dice_metric_fn.aggregate().item()
    metric_batch_test = dice_metric_batch_fn.aggregate()
    metric_tumor = metric_batch_test[0].item()
    metric_lymph = metric_batch_test[1].item()

    print(f"Testing - Avg Dice: {mean_dice_test:.4f}, Tumor Dice: {metric_tumor:.4f}, Lymph Dice: {metric_lymph:.4f}")
    return mean_dice_test, metric_tumor, metric_lymph
    # Save the weights
testing() 

100%|██████████| 36/36 [00:22<00:00,  1.61it/s]


Outputs saved at: /home/nada.saadi/CTPET/hecktor2022_cropped/test_outputs_pt_saved.pt
Testing - Avg Dice: 0.6573, Tumor Dice: 0.6898, Lymph Dice: 0.6491


(0.6572763919830322, 0.6897528767585754, 0.6490647196769714)

In [28]:
data_dir = '/home/nada.saadi/MIS-FM/hecktor2022_cropped'
json_dir_ctpt = "/home/nada.saadi/CTPET/hecktor2022_cropped/MDA_CTPT_TRAIN.json"

In [21]:
import os
import json
import random
from glob import glob

def generate_paths(patient_id):
    base_dir = '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data'
    return {
        'id': os.path.join(base_dir, patient_id),
        'ct': os.path.join(base_dir, patient_id, f"{patient_id}_ct.nii.gz"),
        'pt': os.path.join(base_dir, patient_id, f"{patient_id}_pt.nii.gz"),
        'seg': os.path.join(base_dir, patient_id, f"{patient_id}_gt.nii.gz")
    }

# Assign each data entry to a random fold
all_data = []
num_folds = 5

for file_dir in sorted(glob('data/*')):
    patient_id = file_dir.split('/')[-1]
    # Check if the file belongs to the MDA center
    if patient_id.startswith("MDA-"):
        entry = generate_paths(patient_id)
        entry['fold'] = random.randint(1, num_folds) - 1
        all_data.append(entry)

# Compile data into a JSON structure
data_json = {"training": all_data}

# Save to JSON file
json_file_path = "/home/nada.saadi/CTPET/hecktor2022_cropped/MDA_ctpt_test_new.json"
with open(json_file_path, 'w') as f:
    json.dump(data_json, f, indent=4)

print(f"JSON file created at {json_file_path}")



JSON file created at /home/nada.saadi/CTPET/hecktor2022_cropped/MDA_ctpt_test_new.json


In [22]:
def datafold_read(datalist, basedir, fold=0, key="training"):
    with open(datalist) as f:
        json_data = json.load(f)

    json_data = json_data[key]

    for d in json_data:
        for k in d:
            if isinstance(d[k], list):
                d[k] = [os.path.join(basedir, iv) for iv in d[k]]
            elif isinstance(d[k], str):
                d[k] = os.path.join(basedir, d[k]) if len(d[k]) > 0 else d[k]

    tr = []
    val = []
    for d in json_data:
        if "fold" in d and d["fold"] == fold:
            val.append(d)
        else:
            tr.append(d)

    return tr, val

In [31]:
train_files, validation_files = datafold_read(datalist=json_dir_ctpt, basedir=data_dir, fold=0)
len(train_files), len(validation_files)

(152, 44)

In [23]:
import json

# Create a dictionary with the validation files
validation_data = {"validation": validation_files}

# Save the dictionary to a JSON file
json_file_path = "mda_ctpt_test_json"
with open(json_file_path, 'w') as f:
    json.dump(validation_data, f, indent=4)

print(f"Validation files saved to {json_file_path}")


NameError: name 'validation_files' is not defined

In [24]:
import json

# Path to the JSON file
json_file_path = "mda_ctpt_test_json"

# Read the JSON file
with open(json_file_path, "r") as file:
   ctpt_data = json.load(file)

# Print the first 5 elements
for i in range(5):
    print(ctpt_data["validation"][i])


{'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-001', 'ct': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-001/MDA-001_ct.nii.gz', 'pt': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-001/MDA-001_pt.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-001/MDA-001_gt.nii.gz', 'fold': 0}
{'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-007', 'ct': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-007/MDA-007_ct.nii.gz', 'pt': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-007/MDA-007_pt.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-007/MDA-007_gt.nii.gz', 'fold': 0}
{'id': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-015', 'ct': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-015/MDA-015_ct.nii.gz', 'pt': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-015/MDA-015_pt.nii.gz', 'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-015/MDA-015_gt.nii.gz', 'fold': 0}
{'id': 

In [25]:
len(ctpt_data["validation"])

44

In [26]:
import io

with open(json_dir_ctpt, 'r') as f:
    ctpt_data_data = json.load(f)

NameError: name 'json_dir_ctpt' is not defined

In [36]:
len(ctpt_data["validation"])

44

In [27]:
val_transforms_ctpt = Compose(
    [
        LoadImaged(keys=["ct", "pt", "seg"], ensure_channel_first = True),
        SpatialPadd(keys=["ct", "pt", "seg"], spatial_size=(200, 200, 310), method='end'),
        Orientationd(keys=["ct", "pt", "seg"], axcodes="PLS"),
        NormalizeIntensityd(keys=["pt"]),
        ClipCT(keys=["ct"]),
        ScaleIntensityd(keys=["ct"], minv=0, maxv=1),
        #MulPTFM(keys=["ct","pt"]),
        ConcatItemsd(keys=["pt", "ct"], name="ctpt"),
    ]
)

In [39]:
import torch
from monai.inferers import sliding_window_inference
from monai.metrics import DiceMetric
from monai.transforms import AsDiscrete
from tqdm import tqdm

test_loader = DataLoader(
    dataset=Dataset(data=ctpt_data["validation"], transform=val_transforms_ctpt),
    batch_size=1,  # Batch size for testing can be 1 since no backpropagation is required
    shuffle=False,
    num_workers=4
)
# pt_weights='/home/nada.saadi/CTPET/hecktor2022_cropped/mda-ptonly-PT/ptonly-PT-with4.pth'
# model_ct.load_state_dict(torch.load(pt_weights))
def testing():
    model_ct.eval()
    model_pet.eval()
    dice_metric_fn.reset()
    dice_metric_batch_fn.reset()
    
    # Placeholder for fused outputs
    fused_outputs_list = []

    with torch.no_grad():
        for batch_data in tqdm(test_loader):
            test_inputs_ct, test_labels = batch_data["ct"].cuda(), batch_data["seg"].cuda()
            test_inputs_pt, test_labels = batch_data["pt"].cuda(), batch_data["seg"].cuda()
            # Generate outputs for both CT and PET models
            test_outputs_ct = sliding_window_inference(test_inputs_ct, (96, 96, 96), 4, model_ct)
            test_outputs_pt = sliding_window_inference(test_inputs_pt, (96, 96, 96), 4, model_pet)
            
            # Fuse the outputs
            fused_outputs = 0*test_outputs_ct +  1*test_outputs_pt
            
            # Convert fused outputs to one-hot format for DiceMetric calculation
            fused_outputs_onehot = [AsDiscrete(argmax=True, to_onehot=3)(i) for i in decollate_batch(fused_outputs)]
            test_labels_onehot = [AsDiscrete(to_onehot=3)(i) for i in decollate_batch(test_labels)]
            
            # Append fused outputs for later Dice score calculation
            fused_outputs_list.extend(fused_outputs_onehot)

            # Calculate Dice score for fused output
            dice_metric_fn(y_pred=fused_outputs_onehot, y=test_labels_onehot)
            dice_metric_batch_fn(y_pred=fused_outputs_onehot, y=test_labels_onehot)
            
    # Aggregate the results for the entire dataset
mean_dice_test = dice_metric_fn.aggregate().mean().item()
print(f"Overall Mean Dice Score: {mean_dice_test:.4f}")
metric_batch_test = dice_metric_batch_fn.aggregate()
metric_tumor = metric_batch_test[0].item()
metric_lymph = metric_batch_test[1].item()

# def tensor_to_scalar(tensor):
#     if torch.is_tensor(tensor) and tensor.numel() == 1:
#         return tensor.item()
#     elif not torch.is_tensor(tensor):
#         return tensor  # Assume it's already a scalar if not a tensor
#     else:
#         raise ValueError("The tensor contains more than one element and cannot be converted to a scalar.")
    
    
# mean_dice_test_scalar = tensor_to_scalar(mean_dice_test)
# metric_tumor_scalar = tensor_to_scalar(metric_tumor)
# metric_lymph_scalar = tensor_to_scalar(metric_lymph)


 #Now, you can safely format these values with the format specifier
print(f"Testing - Fused Avg Dice: {mean_dice_test:.4f}, Tumor Dice: {metric_tumor:.4f}, Lymph Dice: {metric_lymph:.4f}")
   

    # Save the fused outputs if needed
# fused_save_path = '/home/nada.saadi/CTPET/hecktor2022_cropped/fused_test_outputs_saved.pt'
# torch.save(fused_outputs_list, fused_save_path)
# print(f"Fused outputs saved at: {fused_save_path}")  
#return mean_dice_test, metric_tumor, metric_lymph

# # Ensure model_ct, model_pet, test_loader, and other necessary variables are correctly defined
testing()


Overall Mean Dice Score: 0.4710
Testing - Fused Avg Dice: 0.4710, Tumor Dice: 0.4293, Lymph Dice: 0.5135


100%|██████████| 44/44 [01:13<00:00,  1.68s/it]


In [30]:
import os
import json
import random
from glob import glob

def generate_paths(patient_id):
    base_dir = '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data'
    return {
        
        'seg': os.path.join(base_dir, patient_id, f"{patient_id}_gt.nii.gz")
    }

# Assign each data entry to a random fold
all_data = []
num_folds = 5

for file_dir in sorted(glob('data/*')):
    patient_id = file_dir.split('/')[-1]
    # Check if the file belongs to the MDA center
    if patient_id.startswith("MDA-"):
        entry = generate_paths(patient_id)
        entry['fold'] = random.randint(1, num_folds) - 1
        all_data.append(entry)

# Compile data into a JSON structure
data_json = {"training": all_data}

# Save to JSON file
json_file_path = "/home/nada.saadi/CTPET/hecktor2022_cropped/mda_labels.json"
with open(json_file_path, 'w') as f:
    json.dump(data_json, f, indent=4)

print(f"JSON file created at {json_file_path}")

JSON file created at /home/nada.saadi/CTPET/hecktor2022_cropped/mda_labels.json


In [31]:
import json

# Path to the JSON file
json_file_path = "/home/nada.saadi/CTPET/hecktor2022_cropped/mda_labels.json"

# Read the JSON file
with open(json_file_path, "r") as file:
    mda_labels = json.load(file)

# Print the first 5 elements
for i in range(5):
    print(mda_labels["training"][i])


{'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-001/MDA-001_gt.nii.gz', 'fold': 2}
{'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-003/MDA-003_gt.nii.gz', 'fold': 3}
{'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-004/MDA-004_gt.nii.gz', 'fold': 1}
{'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-005/MDA-005_gt.nii.gz', 'fold': 2}
{'seg': '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-006/MDA-006_gt.nii.gz', 'fold': 0}


In [32]:
import json

# Path to the JSON file
json_file_path = "/home/nada.saadi/CTPET/hecktor2022_cropped/mda_labels.json"

# Read the JSON file
with open(json_file_path, "r") as file:
    data = json.load(file)

# Calculate the number of elements
num_elements = len(data["training"])

# Print the result
print(f"Number of elements in the JSON file: {num_elements}")


Number of elements in the JSON file: 196


In [33]:
# Path to your JSON file
json_file_path = '/home/nada.saadi/CTPET/hecktor2022_cropped/mda_labels.json'

# Reading the JSON data into a Python dictionary
with open(json_file_path, 'r') as file:
    labels_dict = json.load(file)

In [34]:
# Assuming labels_dict values are the class indices and you know the number of classes
num_classes = 3  # For example, if you have 3 classes: 0, 1, and 2


In [35]:
import os
import shutil

# Define the source directory where the gt_ images are located
source_directory = "/home/nada.saadi/CTPET/hecktor2022_cropped/data"

# Define the destination directory where you want to store the gt_ images
destination_directory = "/home/nada.saadi/CTPET/hecktor2022_cropped/gt_images"

# Create the destination directory if it doesn't exist
os.makedirs(destination_directory, exist_ok=True)

# Iterate through the files in the source directory
for root, dirs, files in os.walk(source_directory):
    # Check if the file is a gt_ image from the MDA center
    for file in files:
        if "MDA" in file and file.endswith("_gt.nii.gz"):
            # Get the full path of the source file
            source_file_path = os.path.join(root, file)
            
            # Get the destination file path by appending the file name to the destination directory
            destination_file_path = os.path.join(destination_directory, file)
            
            # Copy the file to the destination directory
            shutil.copy(source_file_path, destination_file_path)

print("All gt_ images from the MDA center have been stored in the destination directory.")


All gt_ images from the MDA center have been stored in the destination directory.


In [36]:
import torch
import numpy as np
import nibabel as nib

def load_nifti_and_convert_to_onehot(image_path, num_classes):
    """
    Loads a NIfTI segmentation image and converts it to a one-hot encoded tensor.
    
    Args:
    - image_path (str): Path to the NIfTI segmentation image.
    - num_classes (int): Total number of classes.
    
    Returns:
    - torch.Tensor: One-hot encoded tensor of the segmentation with shape [num_classes, D, H, W],
                    where D is the depth of the volume.
    """
    # Load the NIfTI file
    nifti_img = nib.load(image_path)
    
    # Convert the NIfTI file to a numpy array
    image_array = nifti_img.get_fdata()
    
    # Initialize an empty tensor for the one-hot encoding
    one_hot = torch.zeros((num_classes, *image_array.shape), dtype=torch.float32)
    
    # Fill the one-hot tensor
    for c in range(num_classes):
        one_hot[c] = torch.tensor(image_array == c, dtype=torch.float32)
    
    return one_hot

# Example usage
image_path = '/home/nada.saadi/MIS-FM/hecktor2022_cropped/data/MDA-006/MDA-006_gt.nii.gz'
num_classes = 3  # Adjust based on your specific dataset
one_hot_labels = load_nifti_and_convert_to_onehot(image_path, num_classes)


In [37]:
import torch
from monai.metrics import DiceMetric
import os
import json
import nibabel as nib

# Function to load NIfTI segmentation and convert to one-hot encoding (as discussed)
def load_nifti_and_convert_to_onehot(image_path, num_classes):
    nifti_img = nib.load(image_path)
    image_array = nifti_img.get_fdata()
    one_hot = torch.zeros((num_classes, *image_array.shape), dtype=torch.float32)
    for c in range(num_classes):
        one_hot[c] = torch.tensor(image_array == c, dtype=torch.float32)
    return one_hot

# Load the JSON file containing the paths to the ground truth NIfTI images
json_file_path = '/home/nada.saadi/CTPET/hecktor2022_cropped/mda_labels.json'
with open(json_file_path, 'r') as file:
    labels_paths = json.load(file)

# Assuming num_classes is known
num_classes = 3

# Load and convert all ground truth segmentations to one-hot encoding
onehot_labels_list = [load_nifti_and_convert_to_onehot(path, num_classes) for path in labels_paths.values()]

# Stack the list of tensors into a single tensor for batch processing
onehot_labels = torch.stack(onehot_labels_list)

# Load the saved model outputs for CT and PET
outputs_ct = torch.load('/home/nada.saadi/CTPET/hecktor2022_cropped/test_output_ct_saved.pt')
outputs_pt = torch.load('/home/nada.saadi/CTPET/hecktor2022_cropped/test_outputs_pt_saved.pt')

# Define the weights for CT and PET outputs and fuse them
weight_ct = 0.2
weight_pt = 0.8
fused_outputs = weight_ct * outputs_ct + weight_pt * outputs_pt

# Initialize DiceMetric for each class (excluding background)
dice_metric_fn = DiceMetric(include_background=False, reduction="mean_batch")

# Calculate Dice scores using the fused outputs and the one-hot encoded labels
dice_metric_fn.reset()
dice_metric_fn(y_pred=fused_outputs, y=onehot_labels)

metric_batch = dice_metric_fn.aggregate()
metric_tumor = metric_batch[0].item()
metric_lymph = metric_batch[1].item()
mean_dice = sum(metric_batch) / len(metric_batch)

print(f"Fusion - Avg Dice: {mean_dice:.4f}, Tumor Dice: {metric_tumor:.4f}, Lymph Dice: {metric_lymph:.4f}")



TypeError: stat: path should be string, bytes, os.PathLike or integer, not list

In [ ]:
def fuse_masks(mask_ct, mask_pet, w_ct):
    """
    Fuse two segmentation masks by a weighted sum.

    Parameters:
    - mask_ct (torch.Tensor): The segmentation mask from the CT model.
    - mask_pet (torch.Tensor): The segmentation mask from the PET model.
    - w_ct (float): The weight for the CT mask. Must be between 0 and 1.

    Returns:
    - torch.Tensor: The fused segmentation mask.
    """
    w_pet = 1 - w_ct
    return w_ct * mask_ct + w_pet * mask_pet


In [ ]:
# Assuming you have the test outputs of the CT and PET models stored in variables `test_output_ct` and `test_output_pet` respectively
test_outputs_ct = sliding_window_inference(test_inputs, (96, 96, 96), 4, model_pet)
test_outputs_pet = ...

# Apply softmax to the test outputs
test_output_ct_softmax = torch.softmax(test_outputs_ct, dim=1)
test_output_pet_softmax = torch.softmax(test_outputs_pet, dim=1)

# Fuse the test outputs with the specified weights
fused_output = fuse_masks(test_output_ct_softmax, test_output_pet_softmax, w_ct=0.2)

# You can then use the fused output for further analysis or evaluation


NameError: name 'test_outputs_ct' is not defined

In [ ]:
w_ct = 0.2
w_pet = 1 - w_ct
lf=w_ct*test_outputs_ct + w_pet*test_outputs_pt

NameError: name 'test_outputs_ct' is not defined

In [ ]:
# Dummy function for illustration
def load_and_preprocess_image(image_path, target_size=(96, 96, 96), modality='CT'):
    # Implement your loading and preprocessing logic here
    # This might involve reading the image, resampling, normalizing, etc.
    # Return the processed image as a numpy array
    return processed_image

# Load and preprocess the CT and PET images
ct_image_path = '/home/nada.saadi/CTPET/hecktor2022_cropped/data/MDA-183/MDA-183_ct.nii.gz'
pet_image_path = '/home/nada.saadi/CTPET/hecktor2022_cropped/data/MDA-183/MDA-183_pt.nii.gz'

ct_image = load_and_preprocess_image(ct_image_path, modality='CT')
pet_image = load_and_preprocess_image(pet_image_path, modality='PET')

# Convert to PyTorch tensors and add batch dimension if necessary
# Example: ct_image and pet_image are numpy arrays after preprocessing
ct_image_tensor = torch.from_numpy(ct_image).unsqueeze(0).to(device)  # Shape: [1, C, H, W, D]
pet_image_tensor = torch.from_numpy(pet_image).unsqueeze(0).to(device)  # Shape: [1, C, H, W, D]


TypeError: expected np.ndarray (got NoneType)

In [ ]:
with torch.no_grad():  # No need to compute gradients
    # Assuming ct_image and pet_image are tensors of shape [1, C, H, W, D]
    output_ct = ct_model
    output_pet = pet_model

    # Assuming the output masks are logits; apply softmax if needed
    mask_ct = torch.softmax(output_ct, dim=1)
    mask_pet = torch.softmax(output_pet, dim=1)

    # Fuse the masks
    w_ct = 0.2  # Example weight, adjust as needed
    fused_mask = fuse_masks(mask_ct, mask_pet, w_ct)


TypeError: 'str' object is not callable